In [13]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent) +'/')
from utils.visualize import img_grid

from torchvision import transforms 

import numpy as np
import pandas as pd
import torchio as tio
import nibabel as nib

import ipywidgets as ipyw
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
import scipy
from sklearn import metrics
import json 

In [14]:
macaw_path = str(Path(os.getcwd()).parent.parent)
ukbb_path = '/home/erik.ohara/UKBB'
ukbb_T1_warped = '/work/forkert_lab/erik/T1_warped/test'
ukbb_T1_warped_folder = '/work/forkert_lab/erik/T1_warped'
generated_path = '/work/forkert_lab/erik/MACAW/cf_images/2_5X'
output_path = '/work/forkert_lab/erik/MACAW/cf_images/2_5X_temp'
z_initial = 41
z_fim = 140
nsamples = 5

In [15]:
data_path = ukbb_path + '/ukbb_img.csv'
df = pd.read_csv(data_path,low_memory=False)

In [4]:
# Getting the generated images
subjects_eid = np.array([])
cf_age = np.array([])
cf_sex = np.array([])
images_generated = []
files_generated = []
for file in tqdm(os.listdir(generated_path)):
    if '.nii' in file:
        files_generated.append(file)
        splits = file.split("_")
        subject_eid = int(splits[0])
        age = splits[1]
        sex = splits[2].split(".")[0]
        subjects_eid = np.append(subjects_eid,subject_eid)
        cf_age = np.append(cf_age, age)
        cf_sex = np.append(cf_sex, sex)
        image_load = nib.load(os.path.join(generated_path, file)).get_fdata()
        images_generated.append(image_load)
print(len(images_generated))
images_generated = np.array(images_generated)

  0%|          | 0/2370 [00:00<?, ?it/s]

2370


In [5]:
files_with_nan = []
images_with_nan = []

for idx, each_image in tqdm(enumerate(images_generated)):
    if np.isnan(each_image).any():
        images_with_nan.append(each_image)
        files_with_nan.append(files_generated[idx])
print(len(images_with_nan))

print(files_with_nan)

0it [00:00, ?it/s]

28
['1044293_59_Female_16-165.nii.gz', '1122090_73_Male_16-165.nii.gz', '1229016_72_Male_16-165.nii.gz', '1655891_65_Male_16-165.nii.gz', '1702916_56_Male_16-165.nii.gz', '2012450_65_Female_16-165.nii.gz', '2126874_57_Female_16-165.nii.gz', '2228000_70_Female_16-165.nii.gz', '2817908_51_Male_16-165.nii.gz', '3099609_66_Male_16-165.nii.gz', '3231068_69_Female_16-165.nii.gz', '3320348_71_Male_16-165.nii.gz', '3341564_74_Female_16-165.nii.gz', '3411124_80_Female_16-165.nii.gz', '3458141_76_Male_16-165.nii.gz', '3489713_63_Male_16-165.nii.gz', '3517742_72_Female_16-165.nii.gz', '3764165_75_Female_16-165.nii.gz', '3808336_48_Male_16-165.nii.gz', '4081995_75_Male_16-165.nii.gz', '4500278_62_Male_16-165.nii.gz', '4939592_58_Female_16-165.nii.gz', '5018757_77_Male_16-165.nii.gz', '5043977_74_Female_16-165.nii.gz', '5133681_52_Female_16-165.nii.gz', '5145761_78_Male_16-165.nii.gz', '5412747_62_Female_16-165.nii.gz', '5486949_67_Male_16-165.nii.gz']


In [11]:
images_with_nan[0].shape

(150, 150, 100)

In [12]:
for idx, each_image in enumerate(images_with_nan):
    nan_places = np.argwhere(np.isnan(each_image))
    #if len(nan_places)%22500 == 0:
    slices_nan = []
    for nan_position in nan_places:
        x_nan, y_nan, z_nan = nan_position
        if x_nan not in slices_nan:
            slices_nan.append(x_nan)
    print(f"File {files_with_nan[idx]} has {len(nan_places)} positions with zero, and the slices are {slices_nan}")
    #else:
        #print(f"Analyze file {files_with_nan[idx]} in dept since it has {len(nan_places)} positions with zero")

File 1044293_59_Female_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 1122090_73_Male_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 1229016_72_Male_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 1655891_65_Male_16-165.nii.gz has 10088 positions with zero, and the slices are [148]
File 1702916_56_Male_16-165.nii.gz has 10088 positions with zero, and the slices are [148]
File 2012450_65_Female_16-165.nii.gz has 10088 positions with zero, and the slices are [148]
File 2126874_57_Female_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 2228000_70_Female_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 2817908_51_Male_16-165.nii.gz has 10088 positions with zero, and the slices are [148]
File 3099609_66_Male_16-165.nii.gz has 10093 positions with zero, and the slices are [148]
File 3231068_69_Female_16-165.nii.gz has 10093 positions with zero, and the slices

In [16]:
slices_with_problem = np.array([0,148])
slices_with_problem = slices_with_problem + 16

In [17]:
slices_with_problem

array([ 16, 164])

In [19]:
for idx, each_image in enumerate(images_with_nan):
    nan_places = np.argwhere(np.isnan(each_image))
    for nan_position in nan_places:
        x_nan, y_nan, z_nan = nan_position
        if x_nan == 0:
            each_image[x_nan,y_nan,z_nan] = each_image[x_nan+1,y_nan,z_nan]
        else:
            each_image[x_nan,y_nan,z_nan] = (each_image[x_nan-1,y_nan,z_nan]+each_image[x_nan+1,y_nan,z_nan])/2

In [20]:
for idx, each_image in enumerate(images_with_nan):
    nan_places = np.argwhere(np.isnan(each_image))
    print(len(nan_places))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [22]:
for idx, each_image_nan in tqdm(enumerate(images_with_nan)):
    image_load = nib.load(os.path.join(generated_path, files_with_nan[idx]))
    ni_img = nib.Nifti1Image(each_image_nan, image_load.affine, image_load.header)
    nib.save(ni_img, generated_path + "/" + files_with_nan[idx])

0it [00:00, ?it/s]